# Coursera IBM Data Science Capstone Project
## Where to open an Indian restaurant in Ireland?  


By Tamela Maciel, June 2020

This jupyter notebook completes Coursera's IBM Data Science Professional Certificate capstone project.

It uses the Foursquare API, BeautifulSoup for webscraping, and various python libraries to gather location data.

In [18]:
import pandas as pd  #database wrangling
import numpy as np #linear algebra
#import string
from bs4 import BeautifulSoup #webscraping

from pandas import json_normalize # tranform JSON file into a pandas dataframe

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib as mpl
import matplotlib.pyplot as plt
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

import folium # map rendering library

### Get venues for each city using the Foursquare API

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
CLIENT_ID = 'LRZS1Q1R12WGF52OQPOSHLS4CC3NNRZYTSKECAHUTTYW0TMR' # your Foursquare ID
CLIENT_SECRET = 'SYHT1RYKRHP5VV3ARJC01GTXMM0ERHOLZXGPL4TYGOPHS1PB' # your Foursquare Secret
VERSION = '20200601' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: LRZS1Q1R12WGF52OQPOSHLS4CC3NNRZYTSKECAHUTTYW0TMR
CLIENT_SECRET:SYHT1RYKRHP5VV3ARJC01GTXMM0ERHOLZXGPL4TYGOPHS1PB


In [41]:
# return up to 100 restaurants within 1km radius of lat,long. For Leicester this is 90 restaurants.

city = 'Leicester'
lat =52.633331
long = -1.133333
query='restaurant'
radius = 1000
limit = 100

In [42]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    query,
    radius, 
    limit)
results = requests.get(url).json()

'https://api.foursquare.com/v2/venues/explore?&client_id=LRZS1Q1R12WGF52OQPOSHLS4CC3NNRZYTSKECAHUTTYW0TMR&client_secret=SYHT1RYKRHP5VV3ARJC01GTXMM0ERHOLZXGPL4TYGOPHS1PB&v=20200601&ll=52.633331,-1.133333&query=restaurant&radius=1000&limit=100'

In [50]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Peter Pizzeria,Pizza Place,52.631894,-1.133741
1,PizzaExpress,Pizza Place,52.631663,-1.132414
2,Pret A Manger,Sandwich Place,52.635455,-1.132535
3,Pieros Snack Bar,Restaurant,52.634606,-1.135849
4,The White Peacock,Restaurant,52.631138,-1.132235
5,Nando's,Portuguese Restaurant,52.633401,-1.129511
6,Knight & Garter,Gastropub,52.633913,-1.134372
7,Bombay Bites,Indian Restaurant,52.632547,-1.131534
8,Bill's Restaurant,English Restaurant,52.635839,-1.137399
9,Crafty Burger @ St Martin's,Burger Joint,52.634770,-1.134822


In [117]:
# Try for Dublin

city = 'Dublin'
lat =53.349804
long = -6.260310
#query='indian restaurant'
section='food'
#categoryId='4d4b7105d754a06374d81259'
radius = 1500
limit = 100
#offset=100
pop=1

In [118]:
# get venues
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&section={}&radius={}&limit={}&sortByPopularity={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    section,
    radius,
    limit,
    pop)
results = requests.get(url).json()


In [119]:
venues = results['response']['groups'][0]['items']
    
dublin_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
dublin_venues =dublin_venues.loc[:, filtered_columns]

# filter the category for each row
dublin_venues['venue.categories'] = dublin_venues.apply(get_category_type, axis=1)

# clean columns
dublin_venues.columns = [col.split(".")[-1] for col in dublin_venues.columns]

dublin_venues

,name,categories,lat,lng
0,The Brazen Head,Irish Pub,53.344982,-6.276335
1,Brother Hubbard North,Café,53.347137,-6.268377
2,Bread 41,Bakery,53.344812,-6.251619
3,Bunsen,Burger Joint,53.341178,-6.260037
4,Eddie Rocket's,Diner,53.347805,-6.259827
...,...,...,...,...
95,Toffoli,Café,53.343301,-6.269331
96,Bel Cibo,Pizza Place,53.348569,-6.278632
97,Tommy O'Gara's Pub,Irish Pub,53.350993,-6.282447
98,The Sideline Bistro,Restaurant,53.360659,-6.254021


In [120]:
dublin_venues[dublin_venues['categories']=='Indian Restaurant']

,name,categories,lat,lng
81,Ruposhi Indian Restaurant,Indian Restaurant,53.361904,-6.260117
93,Spice n Rice,Indian Restaurant,53.359425,-6.261525


In [121]:
# Try for Cork

city = 'Cork'
lat =51.897869
long = -8.471090
#query='indian restaurant'
section='food'
#categoryId='4d4b7105d754a06374d81259'
radius = 1500
limit = 100
#offset=100
pop=1

In [122]:
# get venues
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&section={}&radius={}&limit={}&sortByPopularity={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    section,
    radius,
    limit,
    pop)
results = requests.get(url).json()


In [123]:
venues = results['response']['groups'][0]['items']
    
cork_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
cork_venues =cork_venues.loc[:, filtered_columns]

# filter the category for each row
cork_venues['venue.categories'] = cork_venues.apply(get_category_type, axis=1)

# clean columns
cork_venues.columns = [col.split(".")[-1] for col in cork_venues.columns]

cork_venues

,name,categories,lat,lng
0,Jackie Lennox's,Fish & Chips Shop,51.892308,-8.483564
1,Son of a Bun,Burger Joint,51.901520,-8.469486
2,Boojum,Burrito Place,51.898421,-8.470647
3,Electric,Gastropub,51.896081,-8.473438
4,Farmgate Café,Café,51.897899,-8.473913
5,Good Day Café,Café,51.893056,-8.472737
6,Market Lane,Restaurant,51.898433,-8.468249
7,Salt Wine Bar,Restaurant,51.896173,-8.457100
8,West Cork Burger Company,Burger Joint,51.897877,-8.476091
9,Ramen Asian Streetfood,Asian Restaurant,51.894211,-8.465214


In [124]:
cork_venues[cork_venues['categories']=='Indian Restaurant']

,name,categories,lat,lng
47,Iyers,Indian Restaurant,51.900993,-8.475838


In [125]:
# Try for Bray

city = 'Bray'
lat =53.204418
long = -6.109190
#query='indian restaurant'
section='food'
#categoryId='4d4b7105d754a06374d81259'
radius = 1500
limit = 100
#offset=100
pop=1

In [126]:
# get venues
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&section={}&radius={}&limit={}&sortByPopularity={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    section,
    radius,
    limit,
    pop)
print(url)
results = requests.get(url).json()

https://api.foursquare.com/v2/venues/explore?&client_id=LRZS1Q1R12WGF52OQPOSHLS4CC3NNRZYTSKECAHUTTYW0TMR&client_secret=SYHT1RYKRHP5VV3ARJC01GTXMM0ERHOLZXGPL4TYGOPHS1PB&v=20200601&ll=53.204418,-6.10919&section=food&radius=1500&limit=100&sortByPopularity=1


In [127]:
venues = results['response']['groups'][0]['items']
    
bray_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
bray_venues =bray_venues.loc[:, filtered_columns]

# filter the category for each row
bray_venues['venue.categories'] = bray_venues.apply(get_category_type, axis=1)

# clean columns
bray_venues.columns = [col.split(".")[-1] for col in bray_venues.columns]

bray_venues

,name,categories,lat,lng
0,Finnbees Coffee,Café,53.203269,-6.097929
1,McDonald's,Fast Food Restaurant,53.200541,-6.111162
2,Butler & Barry,Gastropub,53.203536,-6.098272
3,Box Burger,Burger Joint,53.205306,-6.100271
4,Platform Pizza Bar,Pizza Place,53.205348,-6.100387
5,"Caffè Letterario ""Gatta Nera""",Italian Restaurant,53.203642,-6.100883
6,Dockyard 8,Bistro,53.207533,-6.102158
7,Carpe Diem,Café,53.203026,-6.099403
8,Cassonis,Fish & Chips Shop,53.197602,-6.094455
9,Seaside Fast Food,Fish & Chips Shop,53.198325,-6.094911
